In [14]:
import pandas as pd
import numpy as np
import re
import time
import os
import logging
import vaex

In [15]:
# create a log file
logging.basicConfig(filename=f'log_{time.strftime("%Y%m%d-%H%M%S")}.txt', level=10)
logging.info('Starting Analysis')
logging.info('Read alignment and query inputs')


In [53]:
# read data as vaex dataframe
sequence_alignment = vaex.read_csv('../data/test_3/sample_alignment_3.txt', chunk_size=1000000,convert=True, sep='\t', header=None)
query_sequence = vaex.read_csv('../data/test_4/sample_query_4.txt', chunk_size=10000, convert=True, sep='\t', header=None)


FileNotFoundError: [Errno 2] No such file or directory: '../data/test_4/sample_query_4.txt'

In [51]:
sequence_alignment

#,0,1,2,3
0,TR2456282,chr14,64416601,76M
1,TR2578233,chr14,77744543,72M
2,TR231400,chr2,71753385,76M
3,TR3126806,chrX,32632468,76M
4,TR467682,chr2,179427014,72M
...,...,...,...,...
"38,025,693",TR3065738,chr22,51064028,75M
"38,025,694",TR2136443,chr9,119460446,73M
"38,025,695",TR1601816,chr6,129707187,31M1D45M
"38,025,696",TR2657268,chr15,42695968,72M


In [52]:
query_sequence

#,0,1
0,TR177264,35
1,TR98394,72
2,TR69408,27
3,TR27181,52
4,TR198924,12
5,TR104209,12
6,TR219971,65
7,TR176174,48
8,TR258300,69
9,TR244028,41


In [37]:
def process_query_to_ref_position(cigar_string, ref_start_position, query_index):
    '''To get reference position for a given query sequence index and its cigar string'''
    ref_array = np.array([]).astype('int32')
    query_array = np.array([]).astype('int32')
    query_start_position = 0
    try:
        string_info = re.findall(r'(\d+)(\w)', cigar_string)
    except ValueError as e:
        logging.info(f"Not a valid CIGAR string {cigar_string}")
        logging.exception(e)

    for number, operator in string_info:
        # assess whether the CIGAR operators are valid
        if operator not in ['M', 'I', 'D', 'N', 'S', 'H', 'P', '=', 'X']:
            raise Exception(f"Not a valid CIGAR operator {operator}: \
                            {cigar_string}")

        if operator in ['M', '=', 'X']:
            # present in query and reference
            ref_position = range(
                                    ref_start_position,
                                    ref_start_position+int(number))
            query_position = range(
                                    query_start_position,
                                    query_start_position+int(number))
            ref_array = np.append(ref_array, list(ref_position))
            query_array = np.append(query_array, list(query_position))
            ref_start_position += int(number)
            query_start_position += int(number)
        if operator in ['D', 'N']:
            # present in reference only
            ref_position = range(
                                    ref_start_position,
                                    ref_start_position+int(number))
            query_position = [query_start_position]*int(number)
            ref_array = np.append(ref_array, list(ref_position))
            query_array = np.append(query_array, query_position)
            ref_start_position += int(number)
        if operator in ['I', 'S']:
            # present in query only
            ref_position = [ref_start_position]*int(number)
            query_position = range(
                                    query_start_position,
                                    query_start_position+int(number))
            ref_array = np.append(ref_array, ref_position)
            query_array = np.append(query_array, list(query_position))
            query_start_position += int(number)
        if operator in ['H', 'P']:
            # present in none
            ref_position = [ref_start_position]*int(number)
            query_position = [query_start_position]*int(number)
            ref_array = np.append(ref_array, ref_position)
            query_array = np.append(query_array, query_position)
    logging.info(f'{cigar_string}\t{string_info}\t{ref_array}\t{query_array}')
    
    try:
        len(ref_array) == len(query_array)
    except ValueError as e:
        logging.info(f"Reference array and query array are not equal")
        logging.exception(e)

    
    try:
        ref_position = int(ref_array[list(query_array).index(query_index)])
    except ValueError as e:
        ref_position = -1
        logging.debug(f'Not a valid query position {query_index}')
        logging.exception(e)

    return ref_position

In [38]:
def get_output(tx_id, query_position):
    '''To get desired output by applying the function on query dataframe'''
    try:
        records = sequence_alignment[sequence_alignment['0'] == tx_id]
        if len(records) >= 1:
            logging.debug(f'There are multiple alignments for {tx_id}')
            chrom = records.to_records()[0]['1']
            ref_start = records.to_records()[0]['2']
            cigar_string = records.to_records()[0]['3']
            ref_position = process_query_to_ref_position(cigar_string, ref_start, query_position)
        if len(records) == 0 :
            logging.debug(f'There is no transcript id in alignment {tx_id}')
            chrom = ''
            ref_start = 0
            cigar_string = ''
            ref_position = -1
    except IndexError as e:
        chrom = ''
        ref_position = -1
        logging.info(f'There are no transcripts in alignment for {tx_id}')
    return((chrom,ref_position))

In [46]:
res = query_sequence.apply(get_output, arguments=[query_sequence['0'],query_sequence['1']], vectorize=False)
res = res.evaluate()

In [47]:
res_vx = vaex.from_pandas(pd.DataFrame(res))

In [48]:
query_sequence = query_sequence.join(res_vx,lprefix='left')

In [49]:
query_sequence

#,left0,left1,0,1
0,TR2131,71,chr1,9738412
1,TR2542,41,chr1,12108296
2,TR1788,52,chr1,7587184
3,TR1417,6,chr1,5969898
4,TR1657,66,chr1,6815045
5,TR18,35,chrM,2531
6,TR2880,36,chr1,15170642
7,TR135,10,chr1,536812
8,TR2709,34,chr1,13690241
9,TR2571,46,chr1,12236289


In [13]:
query_sequence.export_csv('test.csv', sep='\t', index=False, header=False)